In [1]:
pip install wikipedia

  Preparing metadata (setup.py) ... - done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=3bb0fd5dca1c5f80ca4b9e502370a1cebb0002a6d634726674305c40ed8ec875
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [2]:
import wikipedia

# Set a custom user agent to comply with Wikipedia's policy
wikipedia.set_user_agent("My Wikipedia Scraping Bot (youremail@example.com)")

# List of Wikipedia page titles
page_titles = [
    "Machine learning",
    "Deep learning",
    "Natural Language Processing", 
    "Recurrent neural network", 
    "Convolutional neural network", 
]

# Dictionary to store page titles and their content
documents = {}

# Number of pages to fetch for each document topic
num_pages_per_topic = 10

# Fetch content for each page
for title in page_titles:
    try:
        # Fetch the page content
        pages = wikipedia.search(title, results=num_pages_per_topic, suggestion=False)

        # Store the title and content in the dictionary
        documents[title] = "\n".join([wikipedia.page(page).content for page in pages])
    except wikipedia.exceptions.PageError:
        print("Page '{}' does not exist.".format(title))
    except wikipedia.exceptions.DisambiguationError as e:
        print("Page '{}' is a disambiguation page. Skipping.".format(title))

# Combine all content into one string
all_content = "\n".join(documents.values())

# Print the combined content
print("Combined Content:")
print(all_content[:500])  # Print only the first 500 characters for demonstration

Page 'Machine learning' does not exist.


WikipediaException: An unknown error occured: "Search is currently too busy. Please try again later.". Please report it on GitHub!

In [ ]:
len(all_content)

In [ ]:
import re
import numpy as np
import nltk
import spacy

from spacy.cli import download
from spacy import load
import warnings

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [ ]:
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet2022')

In [ ]:
class preprocessing:
    def __init__(self,text):
        self.text=text
        self.words=None
    def clear(self):
        clean = re.compile('<.*?>')
        self.text =  re.sub(clean, '', self.text)
        self.text= re.sub(r"[^a-zA-Z]"," ",self.text)
        self.text = re.sub(r'\s+', ' ', self.text)
    def Normalization(self):
        self.text=self.text.lower()
#         print(self.text)
    def Tokenization(self):
        self.words = word_tokenize(self.text)
#         print(self.words)
    def del_stop_words(self):
        stop_words = set(stopwords.words('english'))
        self.words=[word for word in self.words if (word not in stop_words)]
        
    def stemming(self):
        stemmer = PorterStemmer()
        self.words = [stemmer.stem(word) for word in self.words]
    def unique_words(self):
        self.words = list(set(self.words))
        return self.words
    def return_string(self):
        return ' '.join(self.words)
    def del_char (self):
        self.words=[word for word in self.words if (len(word)>1)]
        return self.words

In [ ]:
def clean_document (document):
    cleaner = preprocessing(document)
    cleaner.clear()
    cleaner.Tokenization()
    cleaner.Normalization()
    cleaner.del_stop_words()
    cleaner.stemming()
    cleaner.del_char()
    output = cleaner.return_string()
    return output

In [ ]:
clear = clean_document(all_content)

In [ ]:
clear[:1200]

In [ ]:
words = set(clear.split())  # Unique characters in input text
words_to_indices = dict((c, i) for i, c in enumerate(words))  # Character to index mapping
indices_to_words = dict((i, c) for i, c in enumerate(words))  # Index to character mapping

In [ ]:
indices_to_words

In [ ]:
len(indices_to_words)

In [ ]:
maxlen = 5  # Maximum length of input sequence
step = 3  # Step size for creating input sequences
sentences = []  # List to store input sequences
next_words = []  # List to store next words

In [ ]:
corpus = clear.split()
corpus[:6]

In [ ]:
for i in range(0, len(corpus) - maxlen, step):
    sentences.append(corpus[i:i + maxlen])
    next_words.append(corpus[i + maxlen])


In [ ]:
len(sentences)

In [ ]:
print("X_train : " , sentences[0] , "\nY_train : " , next_words[0])

In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
model = Word2Vec(sentences= sentences, vector_size=100, window=3, min_count=1, workers=4)
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")

In [ ]:
sentences[0]

In [ ]:
word_vectors = model.wv
word_vector_list = []
x=[]
for sentence in sentences:
    sentence_vectors = [word_vectors[word] for word in sentence if word in word_vectors]
    x.append(sentence_vectors)

print(len(x))
print(len(x[0]))

word_vectors = model.wv
word_vector_list = []
y=[]
for sentence in next_words:
    sentence_vectors = [word_vectors[sentence] if sentence in word_vectors else None]
    y.append(sentence_vectors)

print(len(y))
print(len(y[0]))


In [ ]:
y = np.array(y)
x = np.array(x)

In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
x_words = np.zeros((len(sentences), maxlen, len(words)), dtype=bool)  # Initialize input data tensor
y_words = np.zeros((len(sentences), len(words)), dtype=bool)  # Initialize target data tensor

In [ ]:
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        x_words[i, t, words_to_indices[word]] = 1
    y_words[i, words_to_indices[next_words[i]]] = 1

In [ ]:
print(x_words.shape)
print(y_words.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y_words, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf 
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
word_vectors[0].shape[0]

In [ ]:
x[0].shape[0]

In [ ]:
from tensorflow.keras.layers import Dense, SimpleRNN


In [ ]:
import tensorflow as tf

model_rnn = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(128, return_sequences=True, input_shape=(maxlen, word_vectors[0].shape[0])),
    tf.keras.layers.Dropout(0.2),  # Adding dropout after the LSTM layer
    tf.keras.layers.LSTM(128, return_sequences=True),  # Adding LSTM layer with return_sequences=True
    tf.keras.layers.LSTM(128),  # Adding another LSTM layer without return_sequences=True
    tf.keras.layers.Dropout(0.2),  # Adding dropout after the LSTM layer
    tf.keras.layers.Dense(len(words), activation='softmax')
])

model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.summary()


In [ ]:
history = model_rnn.fit(x_train, y_train, epochs=100 , validation_data=(x_test , y_test))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
loss, accuracy = model_rnn.evaluate(x_test,y_test)
print('Testing Accuracy is {} '.format(accuracy*100))